In [ ]:
from pathlib import Path
import random
from ultralytics.models import YOLO
import torch
import yaml

random.seed(42)

In [ ]:
from utils import DisplayPath
Path = DisplayPath

## Step 2: Configure Paths & Hyperparameters

In [ ]:
# Dataset path (created by e2e_data_prep.ipynb)
YOLO_DATASET = Path("datasets/ready/full_dataset")
RUNS_DIR = Path("runs/segment")

# Verify dataset exists
if not YOLO_DATASET.exists():
    raise FileNotFoundError(f"Dataset not found at {YOLO_DATASET}. Run e2e_data_prep.ipynb first!")

print("Dataset:")
YOLO_DATASET.display()
print("  Train:")
(YOLO_DATASET / 'train').display()
print("  Val:")
(YOLO_DATASET / 'val').display()
print("  Test:")
(YOLO_DATASET / 'test').display()

In [ ]:
BATCH_SIZE = 16
IMG_SIZE = 640
model_type = "yolo11n-seg.pt"
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print(f"Device: {DEVICE}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA: {torch.version.cuda}")

## Step 3: Verify Dataset Structure

Dataset is already prepared by e2e_data_prep.ipynb

In [ ]:
# Verify dataset structure
print("="*60)
print("DATASET VERIFICATION")
print("="*60)

splits = ['train', 'val', 'test']
stats = {}

for split in splits:
    img_dir = YOLO_DATASET / split / "images"
    lbl_dir = YOLO_DATASET / split / "labels"
    
    if img_dir.exists() and lbl_dir.exists():
        num_images = len(list(img_dir.glob("*")))
        num_labels = len(list(lbl_dir.glob("*.txt")))
        stats[split] = {'images': num_images, 'labels': num_labels}
        print(f"{split.upper():5s}: {num_images:4d} images, {num_labels:4d} labels")
    else:
        stats[split] = {'images': 0, 'labels': 0}
        print(f"{split.upper():5s}: Missing!")

total_images = sum(s['images'] for s in stats.values())
total_labels = sum(s['labels'] for s in stats.values())

print(f"{'TOTAL':5s}: {total_images:4d} images, {total_labels:4d} labels")
print("="*60)

if total_images == 0:
    raise RuntimeError("No dataset found! Run e2e_data_prep.ipynb to create the dataset.")

## Step 3.5: Analyze Class Distribution

Check the distribution of classes in the training set to identify imbalances

In [ ]:
# Analyze class distribution in training set
from src.data_utils import count_class_instances

print("="*60)
print("CLASS DISTRIBUTION ANALYSIS")
print("="*60)

class_names = {0: 'Red Ball', 1: 'Human', 2: 'Trashcan'}

for split in ['train']:
    counts = count_class_instances(YOLO_DATASET, split)
    total = sum(counts.values())

    print(f"\n{split.upper()}:")
    for class_id, count in counts.items():
        percentage = (count / total * 100) if total > 0 else 0
        print(f"  {class_names[class_id]:12s} (class {class_id}): {count:5d} instances ({percentage:5.1f}%)")
    print(f"  {'TOTAL':12s}           : {total:5d} instances")
    
    # Calculate imbalance ratio
    if counts[2] > 0:  # If trashcans exist
        max_count = max(counts.values())
        min_count = min(v for v in counts.values() if v > 0)
        imbalance_ratio = max_count / min_count
        print(f"  Imbalance ratio: {imbalance_ratio:.1f}x")

print("\n" + "="*60)

In [ ]:
# Import augmentation utilities
import albumentations as A
from src.augmentation import augment_class_dataset, augment_class_with_occlusion

# Define custom augmentation pipelines
trashcan_pipeline = A.Compose([  # type: ignore
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.8),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.8),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.RandomScale(scale_limit=0.3, p=0.7),
    A.Affine(rotate=(-20, 20), translate_percent=0.1, scale=(0.8, 1.2), shear=(-10, 10), p=0.7),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Execute augmentation
DO_AUGMENTATION = True 
if DO_AUGMENTATION:
    # 1. Augment Trashcans (Class 2) - Strong augmentation
    print("Augmenting Trashcans...")
    augment_class_dataset(
        dataset_path=YOLO_DATASET,
        class_id=2,
        num_augmentations=20,
        aug_config=trashcan_pipeline
    )
    
    # 2. Augment Red Balls (Class 0) - Occlusion & Scaling (Hard)
    # This creates smaller balls and balls with holes (occluded), updating labels automatically
    print("Augmenting Red Balls with Occlusion...")
    augment_class_with_occlusion(
        dataset_path=YOLO_DATASET,
        class_id=0,
        num_augmentations=5
    )

### 🔄 Reset: Clean Augmented Images

If you accidentally ran augmentation multiple times, use this to remove all augmented copies and start fresh

In [ ]:
from src.augmentation import clean_augmented_images

# Execute cleanup (set to True to clean)
CLEAN_AUGMENTED = False

if CLEAN_AUGMENTED:
    removed = clean_augmented_images(YOLO_DATASET)
else:
    print("Set CLEAN_AUGMENTED = True to remove all augmented images")

## Step 4: Create YOLO Configuration File

In [ ]:
classes = {
    'red ball': 0,
    'human': 1,
    'trashcan': 2
}

config = {
    'path': str(YOLO_DATASET.absolute()),
    'train': 'train/images',
    'val': 'val/images',
    'nc': len(classes),
    'names': list(classes.keys())
}

config_path = YOLO_DATASET / 'data.yaml'
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print(f"✓ Configuration saved: {config_path}")
print("Dataset structure:")
YOLO_DATASET.display()
print("  Train:")
(YOLO_DATASET / 'train').display()
print("  Val:")
(YOLO_DATASET / 'val').display()
print("  Test:")
(YOLO_DATASET / 'test').display()

## Step 4.5: Select Monitoring Images

Select diverse validation images covering all classes to monitor training progress

In [ ]:
from src.data_utils import select_diverse_monitoring_images

# Select diverse monitoring images
val_labels_dir = YOLO_DATASET / 'val' / 'labels'
val_images_dir = YOLO_DATASET / 'val' / 'images'

MONITOR_IMAGES = select_diverse_monitoring_images(
    val_labels_dir, 
    val_images_dir, 
    images_per_class=3,
    include_mixed=True
)

if len(MONITOR_IMAGES) == 0:
    print("⚠️  Warning: No monitoring images found in validation set!")

## Step 4.6: Define Multi-Class Monitoring Callback

Create a callback that visualizes segmentation progress for all classes at each epoch

In [ ]:
from src.visualization import create_monitoring_callback

## Step 5: Train Model with Multi-Class Monitoring

Train YOLOv11 with:
- Data augmentation on train set (reduced for abundant classes)
- Trashcan pre-augmentation for class balance
- Checkpoints saved for best model
- Validation after each epoch
- **Custom callback to monitor all classes segmentation progress**

In [ ]:
# Load pretrained model
model = YOLO(model_type)

In [ ]:
project_name = 'ball_person_trashcan_model_v5'

In [ ]:
# Setup multi-class monitoring
monitor_output_dir = RUNS_DIR / project_name / 'training_monitor'
print(f"Multi-class monitoring output: {monitor_output_dir}")

# Add callback
callback_fn = create_monitoring_callback(
    model=model,
    monitor_images=MONITOR_IMAGES,
    output_dir=monitor_output_dir,
    project_name=project_name
)

model.add_callback('on_train_epoch_end', callback_fn)
print("✓ Multi-class monitoring callback registered")
print(f"   Monitoring {len(MONITOR_IMAGES)} images covering all classes")

In [ ]:
# Train model
head_idx = next((i for i, m in enumerate(model.model.model) if 'Detect' in m.__class__.__name__ or 'Segment' in m.__class__.__name__), len(model.model.model) - 1)

results = model.train(
    data=str(config_path),
    epochs=10,
    freeze=list(range(head_idx)),
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    device=DEVICE,
    project=str(RUNS_DIR),
    name=project_name,
    exist_ok=True,
    
    # Checkpointing
    save=True,
    save_period=5,  # Save every 5 epochs
    
    # Validation
    val=True,
    
    # Data augmentation
    # **AUG_CONFIG,
    
    # Optimizer
    optimizer='Adam',
    lr0=0.001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    
    # Loss weights - Ajusté pour dataset avec trashcans augmentées
    # Avec l'augmentation massive des trashcans, on peut réduire cls
    box=7.5,
    cls=1.0,      # Réduit de 20.0 à 1.0 car trashcans maintenant bien représentées
    dfl=1.5,
    
    # Other
    patience=20,  # Early stopping
    workers=8,
    verbose=True
)


## Step 6: Load Best Model & Evaluate

In [ ]:
best_model_path = RUNS_DIR / project_name / 'weights' / 'best.pt'
best_model_path.display()
model = YOLO(best_model_path)

## Step 7: Evaluate Results

In [ ]:
# Validation metrics
metrics = model.val()

print("\n" + "="*60)
print("VALIDATION METRICS")
print("="*60)
print(f"Box mAP50: {metrics.box.map50:.4f}")
print(f"Box mAP50-95: {metrics.box.map:.4f}")
print(f"Mask mAP50: {metrics.seg.map50:.4f}")
print(f"Mask mAP50-95: {metrics.seg.map:.4f}")

# Per-class metrics
print("\n" + "="*60)
print("PER-CLASS METRICS (Segmentation)")
print("="*60)
class_names = ['red ball', 'human', 'trashcan']
for i, class_name in enumerate(class_names):
    try:
        map50 = metrics.seg.map50_per_class[i] if hasattr(metrics.seg, 'map50_per_class') else 0
        map_val = metrics.seg.map_per_class[i] if hasattr(metrics.seg, 'map_per_class') else 0
        print(f"{class_name:12s}: mAP50={map50:.4f}, mAP50-95={map_val:.4f}")
    except:
        print(f"{class_name:12s}: metrics not available")
print("="*60)

In [ ]:
# Find best checkpoint
model_dir = RUNS_DIR / project_name
best_model = model_dir / 'weights' / 'best.pt'
last_model = model_dir / 'weights' / 'last.pt'

print(f"Best model: ")
best_model.display()
print(f"Last model: ")
last_model.display()
print(f"Results: ")
model_dir.display()
print(f"\nMulti-class monitoring visualizations: ")
(model_dir / 'training_monitor').display()

## Step 8: Visualize Training Progress Evolution

Review monitoring visualizations showing how segmentation improved for all classes over epochs

In [ ]:
# List all monitoring visualizations
monitor_dir = RUNS_DIR / project_name / 'training_monitor'

if monitor_dir.exists():
    viz_files = sorted(monitor_dir.glob("epoch_*.jpg"))
    print(f"Found {len(viz_files)} monitoring visualizations:")
    for viz_file in viz_files:
        viz_file.display()
    
    if len(viz_files) > 0:
        print(f"\n💡 Tip: Open the images in {monitor_dir} to see how segmentation evolved for all classes!")
        print(f"   You can use an image viewer or VS Code to flip through them chronologically.")
        print(f"   Each image shows detections with: Balls | Humans | Trashcans")
else:
    print("No monitoring visualizations found.")

## Step 9: Test on Sample Images (Optional)

In [ ]:
# Test on validation images (sample from val set)
test_images = list((YOLO_DATASET / "val" / "images").glob("*"))

print(f"Testing on {len(test_images)} sample images...")

for img_path in test_images:
    results = model.predict(str(img_path), save=True, conf=0.1)
    print(f"  ✓ {img_path.name}")

print(f"\nResults saved to:")
(RUNS_DIR / project_name).display()